# Tutorial for basic usage of LLMs

References:
*

In [1]:
%pip install -q huggingface-hub==0.23.0
%pip install -q transformers==4.40.2
%pip install -q accelerate==0.30.1
%pip install -q sentencepiece==0.2.0
%pip install -q datasets==2.19.1
%pip install -q sentence-transformers==2.7.0

In [2]:
import os
import yaml
from google.colab import drive
from getpass import getpass

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read YAML file
f_path = "/content/drive/MyDrive/GitHub/python-codebase/machine_learning/private_keys.yml"
with open(f_path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)
os.environ['HF_API_TOKEN'] = data_loaded['HF_API_KEY']
os.environ['GITHUB_TOKEN'] = data_loaded['GITHUB_TOKEN']

## 1. Prompt the Model - Hosted within HF

In [4]:
import requests

In [9]:
# Example (directly accesing HF endpoint w/ requests)
API_URL = "https://api-inference.huggingface.co/models/gpt2"
headers = {"Authorization": f"Bearer {os.environ['HF_API_TOKEN']}"}
def query(payload):
    try:
      response = requests.post(API_URL, headers=headers, json=payload)
      return response.json()
    except:
      print(requests.post(API_URL, headers=headers, json=payload).text)
      return ""

data = query("Can you please let us know more details about your ")

Failed to deserialize the JSON body into the target type: invalid type: string "Can you please let us know more details about your ", expected struct CompatGenerateRequest at line 1 column 53


In [ ]:
print(data)

[{'generated_text': 'Can you please let us know more details about your _______ (at) gmail.co.kr or (9) 778-5575?\n\nEmail address:'}]


In [11]:
# Example (directly accesing HF endpoint w/ requests)
API_URL = "https://api-inference.huggingface.co/models/bigscience/bloomz-560m"
headers = {"Authorization": f"Bearer {os.environ['HF_API_TOKEN']}"}
def query(payload):
    try:
      response = requests.post(API_URL, headers=headers, json=payload)
      return response.json()
    except:
      print(requests.post(API_URL, headers=headers, json=payload).text)
      return ""

In [16]:
# https://huggingface.co/docs/api-inference/en/detailed_parameters
data = query(
    {
        'inputs': "Can you please let us know more details about your ",
        'parameters': {
            'max_new_tokens': 250,
            'temperature': 0.0
        }
    }
)

In [17]:
print(data)

{'error': 'Model bigscience/bloomz-560m is currently loading', 'estimated_time': 44.73716735839844}


In [ ]:
print(query_input)


Create a JSON object from the information present in this passage: Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718..
Only use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition:
{
  "$defs": {
    "City": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "country": {
          "title": "Country",
          "type": "string"
        },
        "population": {
          "title": "Population",
          "type": "integer"
        }
      },
      "required": [
        "name",
        "country",
        "population"
      ],
      "title": "City",
      "type": "object"
    }
  },
  "properties": {
    "cities": {
      "items": {
        "$ref": "#/$defs/City"
      },
     

In [ ]:
query_input = """
Create a JSON object from the information present in this passage: Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718..
Only use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition:
{
  "$defs": {
    "City": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "country": {
          "title": "Country",
          "type": "string"
        },
        "population": {
          "title": "Population",
          "type": "integer"
        }
      },
      "required": [
        "name",
        "country",
        "population"
      ],
      "title": "City",
      "type": "object"
    }
  },
  "properties": {
    "cities": {
      "items": {
        "$ref": "#/$defs/City"
      },
      "title": "Cities",
      "type": "array"
    }
  },
  "required": [
    "cities"
  ],
  "title": "CitiesData",
  "type": "object"
}
Make sure your response is a dict and not a list.
"""
#data = query(query_input)
data = query(
    {
        'inputs': query_input,
        'parameters': {
            'max_new_tokens': 250,
            'temperature': 0.0
        }
    }
)

In [ ]:
print(data)

[{'generated_text': '\nCreate a JSON object from the information present in this passage: Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France\'s capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718..\nOnly use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition:\n{\n  "$defs": {\n    "City": {\n      "properties": {\n        "name": {\n          "title": "Name",\n          "type": "string"\n        },\n        "country": {\n          "title": "Country",\n          "type": "string"\n        },\n        "population": {\n          "title": "Population",\n          "type": "integer"\n        }\n      },\n      "required": [\n        "name",\n        "country",\n        "population"\n      ],\n      "title": "City",\n      "type": "object"\n    }\n  },\n  "properties": {\n    "cities": {\n      "it

In [ ]:
query_input = '''
[Context]
Create a JSON object from the information like this one:
{
  "$defs": {
    "City": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "country": {
          "title": "Country",
          "type": "string"
        },
        "population": {
          "title": "Population",
          "type": "integer"
        }
      },
      "required": [
        "name",
        "country",
        "population"
      ],
      "title": "City",
      "type": "object"
    }
  },
  "properties": {
    "cities": {
      "items": {
        "$ref": "#/$defs/City"
      },
      "title": "Cities",
      "type": "array"
    }
  },
  "required": [
    "cities"
  ],
  "title": "CitiesData",
  "type": "object"
}

[Request]
Create that JSON object over this information:
Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718..

Only use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition:

Make sure your response is a dict and not a list.

Only include in the generation the JSON for the request information, not all the input query
'''

In [ ]:
# Example (directly accesing HF endpoint w/ requests)
model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
headers = {"Authorization": f"Bearer {os.environ['HF_API_TOKEN']}"}
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [ ]:
data = query(
    {
        'inputs': query_input,
        'parameters': {
            'max_new_tokens': 500,
            'temperature': 0.1
        }
    }
)

In [ ]:
print(data)

[{'generated_text': '\n[Context]\nCreate a JSON object from the information like this one:\n{\n  "$defs": {\n    "City": {\n      "properties": {\n        "name": {\n          "title": "Name",\n          "type": "string"\n        },\n        "country": {\n          "title": "Country",\n          "type": "string"\n        },\n        "population": {\n          "title": "Population",\n          "type": "integer"\n        }\n      },\n      "required": [\n        "name",\n        "country",\n        "population"\n      ],\n      "title": "City",\n      "type": "object"\n    }\n  },\n  "properties": {\n    "cities": {\n      "items": {\n        "$ref": "#/$defs/City"\n      },\n      "title": "Cities",\n      "type": "array"\n    }\n  },\n  "required": [\n    "cities"\n  ],\n  "title": "CitiesData",\n  "type": "object"\n}\n\n[Request]\nCreate that JSON object over this information:\nBerlin is the capital of Germany. It has a population of 3,850,809. Paris, France\'s capital, has 2.161 mill

In [ ]:
print(data[0]['generated_text'].split('Only include in the generation the JSON for the request information, not all the input query')[-1])

 

{
  "cities": [
    {
      "name": "Berlin",
      "country": "Germany",
      "population": 3850809
    },
    {
      "name": "Paris",
      "country": "France",
      "population": 2161000
    },
    {
      "name": "Lisbon",
      "country": "Portugal",
      "population": 504718
    }
  ]
}


### Example for code generation

In [20]:
# Example (directly accesing HF endpoint w/ requests)
model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
headers = {"Authorization": f"Bearer {os.environ['HF_API_TOKEN']}"}
def query(payload):
    try:
      response = requests.post(API_URL, headers=headers, json=payload)
      return response.json()
    except:
      print(requests.post(API_URL, headers=headers, json=payload).text)
      return ""

In [21]:
query_input = "Generate a python script that integrates the library DSPy for LLM models to a model hosted in Hugginface"
data = query(
    {
        'inputs': query_input,
        'parameters': {
            'max_new_tokens': 500,
            'temperature': 0.1
        }
    }
)

In [22]:
print(data)

[{'generated_text': 'Generate a python script that integrates the library DSPy for LLM models to a model hosted in Hugginface.\n\nThe script should be able to:\n\n- Load a model from Hugginface\n- Load a dataset from a CSV file\n- Preprocess the dataset using DSPy\n- Fine-tune the model on the preprocessed dataset\n- Save the fine-tuned model\n\nThe script should have the following structure:\n\n```python\nimport torch\nfrom dspy.preprocessing import TextPreprocessor\nfrom dspy.models import LLMModel\nfrom transformers import AutoTokenizer, AutoModelForSequenceClassification\n\n# Load a model from Hugginface\nmodel_name = "bert-base-uncased"\nmodel = AutoModelForSequenceClassification.from_pretrained(model_name)\ntokenizer = AutoTokenizer.from_pretrained(model_name)\n\n# Load a dataset from a CSV file\ndataset = pd.read_csv("dataset.csv")\n\n# Preprocess the dataset using DSPy\npreprocessor = TextPreprocessor()\nX_preprocessed, y_preprocessed = preprocessor.fit_transform(dataset["text"

In [27]:
print(data[0]['generated_text'])

Generate a python script that integrates the library DSPy for LLM models to a model hosted in Hugginface.

The script should be able to:

- Load a model from Hugginface
- Load a dataset from a CSV file
- Preprocess the dataset using DSPy
- Fine-tune the model on the preprocessed dataset
- Save the fine-tuned model

The script should have the following structure:

```python
import torch
from dspy.preprocessing import TextPreprocessor
from dspy.models import LLMModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load a model from Hugginface
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load a dataset from a CSV file
dataset = pd.read_csv("dataset.csv")

# Preprocess the dataset using DSPy
preprocessor = TextPreprocessor()
X_preprocessed, y_preprocessed = preprocessor.fit_transform(dataset["text"], dataset["label"])

# Fine-tune the model on th

#### Example w/ Llama 3.1

In [41]:
# Example (directly accesing HF endpoint w/ requests)
model_name = 'meta-llama/Meta-Llama-3.1-8B'
API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
headers = {"Authorization": f"Bearer {os.environ['HF_API_TOKEN']}"}
def query(payload):
    try:
      response = requests.post(API_URL, headers=headers, json=payload)
      return response.json()
    except:
      print(requests.post(API_URL, headers=headers, json=payload).text)
      return ""

In [42]:
query_input = "Generate a python script that integrates the library DSPy for LLM models to a model hosted in Hugginface"
data = query(
    {
        'inputs': query_input,
        'parameters': {
            'max_new_tokens': 500,
            'temperature': 0.1
        }
    }
)

In [43]:
print(data)

{'error': 'The model meta-llama/Meta-Llama-3.1-8B is too large to be loaded automatically (16GB > 10GB). Please use Spaces (https://huggingface.co/spaces) or Inference Endpoints (https://huggingface.co/inference-endpoints).'}


#### Prueba

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


# 2. Load the model (on memory) and prompt it

In [ ]:
from transformers import pipeline

In [ ]:
# Define pipeline
pipe = pipeline(
    "text-generation",
    model="bigscience/bloomz-560m",
    device_map="auto"
    )

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
'''
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
'''
print()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
query_input = """
Create a JSON object from the information present in this passage: Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718..
Only use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition:
{
  "$defs": {
    "City": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "country": {
          "title": "Country",
          "type": "string"
        },
        "population": {
          "title": "Population",
          "type": "integer"
        }
      },
      "required": [
        "name",
        "country",
        "population"
      ],
      "title": "City",
      "type": "object"
    }
  },
  "properties": {
    "cities": {
      "items": {
        "$ref": "#/$defs/City"
      },
      "title": "Cities",
      "type": "array"
    }
  },
  "required": [
    "cities"
  ],
  "title": "CitiesData",
  "type": "object"
}
Make sure your response is a dict and not a list.
"""

In [ ]:
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
text_example = """
Only use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition
{
  "$defs": {
    "City": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "country": {
          "title": "Country",
          "type": "string"
        },
        "population": {
          "title": "Population",
          "type": "integer"
        }
      },
      "required": [
        "name",
        "country",
        "population"
      ],
      "title": "City",
      "type": "object"
    }
  },
  "properties": {
    "cities": {
      "items": {
        "$ref": "#/$defs/City"
      },
      "title": "Cities",
      "type": "array"
    }
  },
  "required": [
    "cities"
  ],
  "title": "CitiesData",
  "type": "object"
}
Make sure your response is a dict and not a list.
"""
messages = [
    {
        "role": "system",
        "content": f"Create a JSON object from the information present in the text provided by the user. {text_example}. The information provided by the user is:",
    },
    {"role": "user", "content": "Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718.."},
]

In [ ]:
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


No chat template is defined for this tokenizer - using the default template for the BloomTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



Create a JSON object from the information present in the text provided by the user. 
Only use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition
{
  "$defs": {
    "City": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "country": {
          "title": "Country",
          "type": "string"
        },
        "population": {
          "title": "Population",
          "type": "integer"
        }
      },
      "required": [
        "name",
        "country",
        "population"
      ],
      "title": "City",
      "type": "object"
    }
  },
  "properties": {
    "cities": {
      "items": {
        "$ref": "#/$defs/City"
      },
      "title": "Cities",
      "type": "array"
    }
  },
  "required": [
    "cities"
  ],
  "title": "CitiesData",
  "type": "object"
}
Make sure your response is a dict and not a list.
. The infor

In [ ]:
query_input = """
Classify the sentiment of this text (positive, negative or neutral): "Hello, I'm happy about you". Sentiment:
"""

In [ ]:
chat = [
  {"role": "assistant", "content": ""},
  {"role": "user", "content": "Classify the sentiment of this text (positive, negative or neutral): 'Hello, I'm happy about you'. Sentiment:"},
]

prompt = pipe.tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.1, top_k=10, top_p=0.95)
print(outputs[0]["generated_text"])

</s>Classify the sentiment of this text (positive, negative or neutral): 'Hello, I'm happy about you'. Sentiment:</s>The first time I saw the film I was very excited. I was very excited to see the film.


In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a chatbot that classifies sentiments within texts",
    },
    {
        "role": "user",
        "content": "Classify the sentiment of this text (positive, negative or neutral): 'Hello, I'm happy about you'. Sentiment:"
    },
]

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.1, top_k=10, top_p=0.95)
print(outputs[0]["generated_text"])

You are a chatbot that classifies sentiments within texts</s>Classify the sentiment of this text (positive, negative or neutral): 'Hello, I'm happy about you'. Sentiment:</s>The first time I saw the film I was very excited. I was very excited to see the movie.


In [ ]:
messages = [
    {
        "role": "system",
        "content": "Classify the sentiment of this text (positive, negative or neutral). Only use these categories",
    },
    {
        "role": "user",
        "content": "Sentence: 'Hello, I'm happy about you'. Sentiment:"
    },
]

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.1, top_k=10, top_p=0.95)
print(outputs[0]["generated_text"])

Classify the sentiment of this text (positive, negative or neutral). Only use these categories</s>Sentence: 'Hello, I'm happy about you'. Sentiment:</s>The first time I saw this movie I was very excited. I was very excited to see this movie.


In [ ]:
messages = [
    {
        "role": "system",
        "content": "",
    },
    {
        "role": "user",
        "content": "What's 2+2?"
    },
]

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.1, top_k=10, top_p=0.95)
print(outputs[0]["generated_text"])

</s>What's 2+2?</s>A few days ago, I was in a meeting with a group of people who were talking about the future of the world. They were talking about the future of the world.


In [ ]:
# Using GPT2

# Define pipeline
pipe2 = pipeline(
    "text-generation",
    model="openai-community/gpt2",
    device_map="auto"
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are an expert on math. Only provide the results of the operations. No explanations or additional text.",
    },
    {"role": "user", "content": "If 1+1 is 2, what's 2+2?"},
]
prompt = pipe2.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
outputs = pipe2(prompt, max_new_tokens=10, do_sample=True, temperature=0.1, top_k=10, top_p=0.95)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


You are an expert on math. Only provide the results of the operations. No explanations or additional text.<|endoftext|>If 1+1 is 2, what's 2+2?<|endoftext|>The following is a list of the most common questions


In [ ]:
messages = [
    {
        "role": "system",
        "content": "Classify the sentiment of this text (positive, negative or neutral). Only use these categories",
    },
    {
        "role": "user",
        "content": "Sentence: 'Hello, I'm happy about you'. Sentiment:"
    },
]

prompt = pipe2.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
outputs = pipe2(prompt, max_new_tokens=100, do_sample=True, temperature=0.1, top_k=10, top_p=0.95)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Classify the sentiment of this text (positive, negative or neutral). Only use these categories<|endoftext|>Sentence: 'Hello, I'm happy about you'. Sentiment:<|endoftext|>The first time I saw the new Star Wars movie, I was so excited. I was so excited to see the new Star Wars movie. I was so excited to see the new Star Wars movie. I was so excited to see the new Star Wars movie. I was so excited to see the new Star Wars movie. I was so excited to see the new Star Wars movie. I was so excited to see the new Star Wars movie. I was so excited to see the new Star Wars movie.


# 3. Using the foundational model (in memory) directly (no chat)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        # temperature=0.2,
        # top_p=0.95,
        # do_sample=True,
        repetition_penalty=1.5,  # Avoid repetition.
        early_stopping=True,  # The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id,
    )
    return outputs

In [ ]:
model_name = "bigscience/bloomz-560m"
# model_name="bigscience/bloom-1b1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
query_input = """
Create a JSON object from the information present in this passage: Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France's capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718..
Only use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition:
{
  "$defs": {
    "City": {
      "properties": {
        "name": {
          "title": "Name",
          "type": "string"
        },
        "country": {
          "title": "Country",
          "type": "string"
        },
        "population": {
          "title": "Population",
          "type": "integer"
        }
      },
      "required": [
        "name",
        "country",
        "population"
      ],
      "title": "City",
      "type": "object"
    }
  },
  "properties": {
    "cities": {
      "items": {
        "$ref": "#/$defs/City"
      },
      "title": "Cities",
      "type": "array"
    }
  },
  "required": [
    "cities"
  ],
  "title": "CitiesData",
  "type": "object"
}
Make sure your response is a dict and not a list.
"""

In [ ]:
input_prompt = tokenizer(
    query_input,
    return_tensors="pt"
)
foundational_outputs_prompt = get_outputs(
    foundational_model,
    input_prompt,
    max_new_tokens=1000
)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


['\nCreate a JSON object from the information present in this passage: Berlin is the capital of Germany. It has a population of 3,850,809. Paris, France\'s capital, has 2.161 million residents. Lisbon is the capital and the largest city of Portugal with the population of 504,718..\nOnly use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition:\n{\n  "$defs": {\n    "City": {\n      "properties": {\n        "name": {\n          "title": "Name",\n          "type": "string"\n        },\n        "country": {\n          "title": "Country",\n          "type": "string"\n        },\n        "population": {\n          "title": "Population",\n          "type": "integer"\n        }\n      },\n      "required": [\n        "name",\n        "country",\n        "population"\n      ],\n      "title": "City",\n      "type": "object"\n    }\n  },\n  "properties": {\n    "cities": {\n      "items": {\n        "$

In [ ]:
query_input = """
Classify the sentiment of this text (positive, negative or neutral): "Hello, I'm happy about you". Sentiment:
"""

In [ ]:
input_prompt = tokenizer(
    query_input,
    return_tensors="pt"
)
foundational_outputs_prompt = get_outputs(
    foundational_model,
    input_prompt,
    max_new_tokens=1000
)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


['\nClassify the sentiment of this text (positive, negative or neutral): "Hello, I\'m happy about you". Sentiment: \n Positive']
